In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow.keras import layers, models

# Încarcă dataset-ul
df = pd.read_csv("ev_charging_synthetic_data.csv")

# Caracteristicile (input) și ținta (output)
# Vom folosi 'Distance Driven (since last charge)' și 'Energy Consumed (kWh)' ca variabile de intrare, dar poți ajusta după necesități.
X = df.drop(columns=[ "Charging Cost (USD)", "State of Charge (Start %)", "State of Charge (End %)"])

# Definim ținta ca 'Charging Duration (hours)' (Timpul până la următoarea încărcare, ce poate fi estimat din durata încărcării)
y = df["Charging Duration (hours)"]

# Împărțirea datelor în seturi de antrenament și testare
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizare caracteristici
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crearea modelului MLP
model = models.Sequential([
    layers.InputLayer(input_shape=(X_train_scaled.shape[1],)),  # Numărul de features
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)  # Model de regresie, deci doar un singur neuron în output
])

# Compilarea modelului
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Antrenarea modelului
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluarea modelului
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.2f} ore")

# Salvăm modelul antrenat
model.save("mlp_ev_charging_model.h5")


C:\Users\adelg\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 37.5076 - mae: 4.2923 - val_loss: 35.2401 - val_mae: 4.2935
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 35.8608 - mae: 4.1067 - val_loss: 27.5032 - val_mae: 3.6753
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 23.8576 - mae: 3.4520 - val_loss: 18.2199 - val_mae: 2.9246
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 16.3459 - mae: 2.7972 - val_loss: 9.5677 - val_mae: 2.3878
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 9.1481 - mae: 2.3078 - val_loss: 5.6155 - val_mae: 1.8627
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 5.1078 - mae: 1.7245 - val_loss: 3.1086 - val_mae: 1.2995
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.2515 - mae: 1.0613 - val_loss: 1.7436 - val_mae: 0.9770
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.0967 - mae: 0.7421 - val_loss: 1.1859 - val_mae: 0.8483
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.8915

Mean Absolute Error (MAE): 0.45 ore


In [3]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# Încarcă modelul cu custom_objects pentru a remedia eroarea
model = tf.keras.models.load_model("mlp_ev_charging_model.h5", custom_objects={'mse': tf.keras.losses.MeanSquaredError()})

# Încarcă datele de testare
test_data = pd.read_csv("ev_charging_patterns_data_encoded.csv")

# Caracteristicile de intrare - asigură-te că ai aceleași coloane ca la antrenare
X_test = test_data.drop(columns=["User ID", "Charging Start Time", "Charging End Time", "Charging Station ID", "Charging Cost (USD)", "State of Charge (Start %)", "State of Charge (End %)"])

# Aplică aceeași standardizare folosită la antrenare
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)  # Ai nevoie de același scaler folosit la antrenare (dacă l-ai salvat)

# Prezicerea energiei consumate pentru fiecare test
predictions = model.predict(X_test_scaled)

# Definirea consumului mediu pe kilometru (kWh/km)
consumption_per_km = 1 / 10  # 1 kWh pentru 10 kilometri, adică 0.1 kWh/km

# Calcularea distanței estimate până la următoarea încărcare
for i, pred in enumerate(predictions):
    # Estimăm distanța până la următoarea încărcare pe baza energiei disponibile în baterie
    energy_left_in_battery = (test_data.loc[i, "State of Charge (End %)"] / 100) * test_data.loc[i, "Battery Capacity (kWh)"]  # Energia rămasă în baterie (kWh)
    
    # Calculăm distanța estimată până la următoarea încărcare
    estimated_distance = energy_left_in_battery / consumption_per_km  # Distanța estimată în kilometri
    
    # Calculăm distanța totală pe care o poate parcurge vehiculul cu bateria complet încărcată
    total_distance = (test_data.loc[i, "Battery Capacity (kWh)"] / consumption_per_km)  # Distanța totală posibilă cu bateria complet încărcată
    
    # Afișăm rezultatele pe un singur rând
    print(f"Testare {i+1} - Distanța estimată: {estimated_distance:.2f} km | Distanța cu bateria complet încărcată: {total_distance:.2f} km")


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 31, but received input with shape (32, 25)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 25), dtype=float32)
  • training=False
  • mask=None

In [16]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# Încarcă modelul cu custom_objects pentru a remedia eroarea
model = tf.keras.models.load_model("mlp_ev_charging_model.h5", custom_objects={'mse': tf.keras.losses.MeanSquaredError()})

# Încarcă datele de testare (pentru a folosi structura de date)
test_data = pd.read_csv("ev_charging_patterns_data_encoded.csv")

# Caracteristicile de intrare (același proces ca la antrenare)
X_test = test_data.drop(columns=["User ID", "Charging Start Time", "Charging End Time", "Charging Station ID", "Charging Cost (USD)", "State of Charge (Start %)", "State of Charge (End %)"])

# Aplică aceeași standardizare folosită la antrenare
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Prezicerea energiei consumate pentru fiecare test
predictions = model.predict(X_test_scaled)

# Definirea consumului mediu pe kilometru (kWh/km)
consumption_per_km = 1 / 10  # 1 kWh pentru 10 kilometri, adică 0.1 kWh/km

# Funcție pentru a prezice distanțele pe baza stării bateriei și a predicțiilor
def predict_distance(state_of_charge_percent, battery_capacity_kWh, vehicle_age, outside_temperature, road_type):
    # Ajustăm consumul pe kilometru în funcție de vârsta vehiculului
    age_factor = 1 + (vehicle_age * 0.02)  # Crește consumul cu 2% pentru fiecare an de vechime

    # Ajustăm consumul pe kilometru în funcție de temperatura exterioară
    if outside_temperature < 0:  # Temperaturile sub 0°C scad eficiența
        temp_factor = 1.2  # Crește consumul cu 20% în condiții de temperaturi scăzute
    elif outside_temperature > 30:  # Temperaturile foarte ridicate (peste 30°C) pot reduce eficiența
        temp_factor = 1.15  # Crește consumul cu 15% în condiții de temperaturi ridicate
    else:
        temp_factor = 1  # Nu se face ajustare în condiții de temperaturi moderate

    # Ajustăm consumul pe kilometru în funcție de tipul de drum
    if road_type == 'Autostrada':
        road_factor = 0.9  # Consum mai mic pe autostradă
    elif road_type == 'Oras':
        road_factor = 1.2  # Consum mai mare în oraș
    elif road_type == 'Munti':
        road_factor = 1.5  # Consum mult mai mare la munte
    else:
        road_factor = 1  # Default, pentru drumuri necunoscute

    # Calculăm energia rămasă în baterie
    energy_left_in_battery = (state_of_charge_percent / 100) * battery_capacity_kWh  # Energia rămasă în baterie (kWh)
    
    # Calculăm distanța estimată până la următoarea încărcare ajustată pentru vârsta vehiculului, temperatura exterioară și tipul de drum
    estimated_distance = energy_left_in_battery / (consumption_per_km * age_factor * temp_factor * road_factor)  # Distanța estimată în kilometri
    
    # Calculăm distanța totală pe care o poate parcurge vehiculul cu bateria complet încărcată
    total_distance = (battery_capacity_kWh / (consumption_per_km * age_factor * temp_factor * road_factor))  # Distanța totală posibilă cu bateria complet încărcată
    
    # Calculăm durata estimată pentru a parcurge distanța estimată
    # Estimăm o viteză medie de 80 km/h pentru autostradă și 50 km/h pentru oraș, iar la munte 40 km/h
    if road_type == 'Autostrada':
        avg_speed = 100  # Viteză pe autostradă
    elif road_type == 'Oras':
        avg_speed = 50  # Viteză în oraș
    elif road_type == 'Munti':
        avg_speed = 40  # Viteză la munte
    else:
        avg_speed = 60  # Viteză medie standard

    # Calculăm durata estimată pentru a ajunge la următoarea încărcare
    estimated_duration = estimated_distance / avg_speed  # Durata în ore

    # Calculăm viteza de descărcare a bateriei (kWh pe oră)
    battery_drain_rate = energy_left_in_battery / estimated_duration  # Viteza de descărcare în kWh pe oră

    return estimated_distance, total_distance, estimated_duration, battery_drain_rate

# Input de la utilizator
state_of_charge = float(input("Introduceți starea bateriei (în procente): "))
battery_capacity = float(input("Introduceți capacitatea bateriei vehiculului (în kWh): "))
vehicle_age = float(input("Introduceți vârsta vehiculului (în ani): "))
outside_temperature = float(input("Introduceți temperatura exterioară (în grade Celsius): "))
road_type = input("Introduceți tipul de drum ('Autostrada', 'Oras', 'Munti'): ")

# Calculăm distanțele și durata pentru starea bateriei introduse
estimated_distance, total_distance, estimated_duration, battery_drain_rate = predict_distance(state_of_charge, battery_capacity, vehicle_age, outside_temperature, road_type)

# Afișăm rezultatele
print(f"Distanța estimată până la următoarea încărcare: {estimated_distance:.2f} km | Distanța totală cu bateria complet încărcată: {total_distance:.2f} km")
print(f"Durata estimată până la următoarea încărcare: {estimated_duration:.2f} ore | Viteza de descărcare a bateriei: {battery_drain_rate:.2f} kWh/oră")



42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


Introduceți starea bateriei (în procente):  25
Introduceți capacitatea bateriei vehiculului (în kWh):  54
Introduceți vârsta vehiculului (în ani):  2
Introduceți temperatura exterioară (în grade Celsius):  5
Introduceți tipul de drum ('Autostrada', 'Oras', 'Munti'):  Munti


Distanța estimată până la următoarea încărcare: 86.54 km | Distanța totală cu bateria complet încărcată: 346.15 km
Durata estimată până la următoarea încărcare: 2.16 ore | Viteza de descărcare a bateriei: 6.24 kWh/oră


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import tensorflow as tf
import os
from tensorflow.keras import layers, models
import numpy as np

# --- Încarcă dataset ---
df = pd.read_csv("ev_charging_synthetic_data.csv")

# --- Calculează țintele noi ---
df['Estimated Total Range (km)'] = df['Battery Capacity (kWh)'] * df['Distance per kWh']
df['Km Remaining'] = df['Estimated Total Range (km)'] - df['Distance Driven (since last charge) (km)']
df['Energy Remaining (kWh)'] = (df['State of Charge (End %)'] / 100) * df['Battery Capacity (kWh)']
average_consumption_per_hour = df['Charging Efficiency (kWh/h)'].mean()
df['Hours Remaining'] = df['Energy Remaining (kWh)'] / average_consumption_per_hour

features = df.drop(columns=[
    'Charging Cost (USD)', 'Charging Duration (hours)', 'Km Remaining', 'Hours Remaining',
    'Estimated Total Range (km)', 'Energy Remaining (kWh)'
]).columns.tolist()

X = df[features]
y_km = df['Km Remaining']
y_hours = df['Hours Remaining']

X_train, X_test, y_km_train, y_km_test, y_hours_train, y_hours_test = train_test_split(
    X, y_km, y_hours, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Km Remaining
model_km = models.Sequential([
   # layers.InputLayer(input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(64, activation='relu',input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])
model_km.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_km.fit(X_train_scaled, y_km_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Model Hours Remaining
model_hours = models.Sequential([
   # layers.InputLayer(input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(64, activation='relu',input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])
model_hours.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_hours.fit(X_train_scaled, y_hours_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Predicții
km_pred = model_km.predict(X_test_scaled).flatten()
hours_pred = model_hours.predict(X_test_scaled).flatten()

import joblib
joblib.dump(scaler, "modele/scaler_km_h.pkl")
os.makedirs("modele", exist_ok=True)

# Salvează modelul pentru predicția Km Remaining
model_km.save("modele/seq_layers_km_model.h5")

# Salvează modelul pentru predicția Hours Remaining (dacă vrei și acesta)
model_hours.save("modele/seq_layers_hours_model.h5")
# MAE
print(f"MAE Km : {mean_absolute_error(y_km_test, km_pred):.2f} km")
print(f"MAE Hours : {mean_absolute_error(y_hours_test, hours_pred):.2f} ore")

# R²
print(f"R² Km : {r2_score(y_km_test, km_pred):.4f}")
print(f"R² Hours : {r2_score(y_hours_test, hours_pred):.4f}")

# Funcție predictie
def estimate_remaining(input_data):
    input_df = pd.DataFrame([input_data], columns=features)
    input_scaled = scaler.transform(input_df)
    km_rem = model_km.predict(input_scaled)[0,0]
    hours_rem = model_hours.predict(input_scaled)[0,0]
    return round(km_rem, 2), round(hours_rem, 2)

# Exemplu input
example_input = X.iloc[0].to_dict()

km_remaining_pred, hours_remaining_pred = estimate_remaining(example_input)
print(f"Km rămași estimați: {km_remaining_pred} km")
print(f"Ore rămase estimate: {hours_remaining_pred} ore")


C:\Users\adelg\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\adelg\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


MAE Km : 17.18 km
MAE Hours : 0.15 ore
R² Km : 0.9006
R² Hours : 0.9150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Km rămași estimați: 270.510009765625 km
Ore rămase estimate: 2.259999990463257 ore


In [3]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np

# --- Încarcă modelul salvat ---
model = tf.keras.models.load_model("mlp_ev_charging_model.h5", custom_objects={'mse': tf.keras.losses.MeanSquaredError()})

# --- Încarcă datele pentru scaler ---
df = pd.read_csv("ev_charging_synthetic_data.csv")

# --- Features relevante pentru estimarea km și ore rămase ---
features = [
    "Battery Capacity (kWh)",
    "Charging Rate (kW)",
    "Time of Day",
    "Day of Week",
    "State of Charge (Start %)",
    "State of Charge (End %)",
    "Distance Driven (since last charge) (km)",
    "Temperature (°C)",
    "Vehicle Age (years)",
    "Vehicle Model_BMW i3",
    "Vehicle Model_Chevy Bolt",
    "Vehicle Model_Hyundai Kona",
    "Vehicle Model_Nissan Leaf",
    "Vehicle Model_Tesla Model 3",
    "Charging Station Location_Chicago",
    "Charging Station Location_Houston",
    "Charging Station Location_Los Angeles",
    "Charging Station Location_New York",
    "Charging Station Location_San Francisco",
    "User Type_Casual Driver",
    "User Type_Commuter",
    "User Type_Long-Distance Traveler",
    "Charger Type_DC Fast Charger",
    "Charger Type_Level 1",
    "Charger Type_Level 2",
    "Charging Efficiency (kWh/h)",
    "Energy per Charge %",
    "Distance per kWh",
    "Total Charge Gained",
    "Charger Efficiency",
    "Temperature Adjusted Consumption"
]


# --- Creează și potrivește scalerul ---
X = df[features]
scaler = StandardScaler()
scaler.fit(X)

# --- Funcție pentru prezicerea energiei consumate folosind modelul ---
def predict_energy_consumed(input_data):
    input_df = pd.DataFrame([input_data], columns=features)
    input_scaled = scaler.transform(input_df)
    predicted_energy = model.predict(input_scaled)[0, 0]
    return predicted_energy

# --- Funcție pentru estimarea distanței și timpului rămas ---
def predict_distance_and_time(state_of_charge_percent, battery_capacity_kWh, vehicle_age, outside_temperature, road_type, input_data_for_model):
    predicted_energy_consumed = predict_energy_consumed(input_data_for_model)

    base_consumption_per_km = 0.1  # 1 kWh pentru 10 km

    age_factor = 1 + (vehicle_age * 0.02)
    if outside_temperature < 0:
        temp_factor = 1.2
    elif outside_temperature > 30:
        temp_factor = 1.15
    else:
        temp_factor = 1.0

    if road_type == 'Autostrada':
        road_factor = 0.9
        avg_speed = 100
    elif road_type == 'Oras':
        road_factor = 1.2
        avg_speed = 50
    elif road_type == 'Munti':
        road_factor = 1.5
        avg_speed = 40
    else:
        road_factor = 1.0
        avg_speed = 60

    adjusted_consumption_per_km = base_consumption_per_km * age_factor * temp_factor * road_factor

    energy_left = (state_of_charge_percent / 100) * battery_capacity_kWh

    estimated_distance = energy_left / adjusted_consumption_per_km

    total_distance = battery_capacity_kWh / adjusted_consumption_per_km

    estimated_duration = estimated_distance / avg_speed

    battery_drain_rate = energy_left / estimated_duration if estimated_duration != 0 else 0

    return {
        "Predicted Energy Consumed (kWh) urm ciclu de utilizare": predicted_energy_consumed,
        "Estimated Distance Remaining (km)": estimated_distance,
        "Total Possible Distance (km) bateria incarcata": total_distance,
        "Estimated Time Remaining (hours)": estimated_duration,
        "Consum pe H (kWh/h)": battery_drain_rate
    }

# --- Exemplu de input ---
def prepare_features_from_user_input(user_input):
    """
    Primește un dict cu input simplu de la utilizator, calculează variabilele derivate
    și returnează dict-ul complet pentru model.
    """
    energy = user_input["Energy Consumed (kWh)"] if "Energy Consumed (kWh)" in user_input else None
    rate = user_input["Charging Rate (kW)"]
    soc_start = user_input["State of Charge (Start %)"]
    soc_end = user_input["State of Charge (End %)"]
    dist = user_input["Distance Driven (since last charge) (km)"]
    temp = user_input["Temperature (°C)"]

    if energy is None:
        # Dacă utilizatorul nu a introdus consumul, estimăm o valoare (ex: folosind Charging Efficiency medie)
        # Pentru exemplu, îl setăm ca 20 kWh (poți ajusta)
        energy = 20

    charge_diff = soc_end - soc_start
    if charge_diff == 0:
        charge_diff = 1e-6  # evităm împărțirea la zero

    features = user_input.copy()

    # Calcule derivate
    features["Charging Efficiency (kWh/h)"] = energy / rate if rate != 0 else 0
    features["Energy per Charge %"] = energy / charge_diff
    features["Distance per kWh"] = dist / energy if energy != 0 else 0
    features["Total Charge Gained"] = charge_diff
    features["Charger Efficiency"] = rate / charge_diff
    features["Temperature Adjusted Consumption"] = energy * (1 + abs(temp - 20) / 20)

    return features


# Exemplu input simplu de la utilizator (fără variabile calculate)
user_input = {
    "Battery Capacity (kWh)": 50,
    "Charging Rate (kW)": 34,
    "Time of Day": 14,
    "Day of Week": 3,
    "State of Charge (Start %)": 60,
    "State of Charge (End %)": 80,
    "Distance Driven (since last charge) (km)": 120,
    "Temperature (°C)": 18,
    "Vehicle Age (years)": 5,

    "Vehicle Model_BMW i3": 0,
    "Vehicle Model_Chevy Bolt": 0,
    "Vehicle Model_Hyundai Kona": 0,
    "Vehicle Model_Nissan Leaf": 0,
    "Vehicle Model_Tesla Model 3": 1,

    "Charging Station Location_Chicago": 0,
    "Charging Station Location_Houston": 1,
    "Charging Station Location_Los Angeles": 0,
    "Charging Station Location_New York": 0,
    "Charging Station Location_San Francisco": 0,

    "User Type_Casual Driver": 1,
    "User Type_Commuter": 0,
    "User Type_Long-Distance Traveler": 0,

    "Charger Type_DC Fast Charger": 1,
    "Charger Type_Level 1": 0,
    "Charger Type_Level 2": 0
}

full_features = prepare_features_from_user_input(user_input)
# --- Input utilizator ---
state_of_charge = 100
battery_capacity = 50
vehicle_age = 1
outside_temperature = 18
road_type = 'Oras'
# --- Coloanele folosite la antrenare ---
features_train = df.columns.drop(['Charging Duration (hours)', 'Charging Cost (USD)'], errors='ignore').tolist()
print(f"Număr features train: {len(features_train)}")
print(f"Features train: {features_train}")

# --- Apel funcție de predicție ---
result = predict_distance_and_time(state_of_charge, battery_capacity, vehicle_age, outside_temperature, road_type, full_features)

# --- Afișare rezultat ---
for k, v in result.items():
    print(f"{k}: {v:.2f}")
# Predicted Energy Consumed (kWh) urm ciclu de utilizare: 3.88
# Estimated Distance Remaining (km): 408.50
# Total Possible Distance (km) bateria incarcata: 408.50
# Estimated Time Remaining (hours): 8.17
# Consum pe H (kWh/h): 6.12

Număr features train: 32
Features train: ['Battery Capacity (kWh)', 'Energy Consumed (kWh)', 'Charging Rate (kW)', 'Time of Day', 'Day of Week', 'State of Charge (Start %)', 'State of Charge (End %)', 'Distance Driven (since last charge) (km)', 'Temperature (°C)', 'Vehicle Age (years)', 'Vehicle Model_BMW i3', 'Vehicle Model_Chevy Bolt', 'Vehicle Model_Hyundai Kona', 'Vehicle Model_Nissan Leaf', 'Vehicle Model_Tesla Model 3', 'Charging Station Location_Chicago', 'Charging Station Location_Houston', 'Charging Station Location_Los Angeles', 'Charging Station Location_New York', 'Charging Station Location_San Francisco', 'User Type_Casual Driver', 'User Type_Commuter', 'User Type_Long-Distance Traveler', 'Charging Efficiency (kWh/h)', 'Energy per Charge %', 'Distance per kWh', 'Total Charge Gained', 'Charger Efficiency', 'Temperature Adjusted Consumption', 'Charger Type_DC Fast Charger', 'Charger Type_Level 1', 'Charger Type_Level 2']


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Predicted Energy Consumed (kWh) urm ciclu de utilizare: 3.88
Estimated Distance Remaining (km): 408.50
Total Possible Distance (km) bateria incarcata: 408.50
Estimated Time Remaining (hours): 8.17
Consum pe H (kWh/h): 6.12


In [13]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib
# === Funcție suplimentară folosind modelele salvate ===
def predict_km_and_hours_remaining(input_data):
    features = list(input_data.keys())
    
    # Schimbă locul celor două coloane în listă
    # if 'Charging Rate (kW)' in features and 'Energy Consumed (kWh)' in features:
    #     idx_rate = features.index('Charging Rate (kW)')
    #     idx_energy = features.index('Energy Consumed (kWh)')
        
    #     # Swap
    #     features[idx_rate], features[idx_energy] = features[idx_energy], features[idx_rate]
    
    # Creează DataFrame cu noua ordine
    #input_df = pd.DataFrame([input_data], columns=features)

    #print("predict km and hours shape", input_data.columns)
    input_scaled = scaler_km_hours.transform(input_data)

    predicted_km = model_km.predict(input_scaled)[0, 0]
    predicted_hours = model_hours.predict(input_scaled)[0, 0]

    return round(predicted_km, 2), round(predicted_hours, 2)

# === Estimare logică distanță și timp ===
def predict_distance_and_time(state_of_charge_percent, battery_capacity_kWh, vehicle_age, outside_temperature, road_type, input_data_for_model):
    # predicted_energy_consumed = predict_energy_consumed(input_data_for_model)

    base_consumption_per_km = 0.1
    age_factor = 1 + (vehicle_age * 0.02)
    temp_factor = 1.2 if outside_temperature < 0 else 1.15 if outside_temperature > 30 else 1.0

    road_factor = {"Autostrada": 0.9, "Oras": 1.2, "Munti": 1.5}.get(road_type, 1.0)
    avg_speed = {"Autostrada": 100, "Oras": 50, "Munti": 40}.get(road_type, 60)

    adjusted_consumption_per_km = base_consumption_per_km * age_factor * temp_factor * road_factor
    energy_left = (state_of_charge_percent / 100) * battery_capacity_kWh

    estimated_distance = energy_left / adjusted_consumption_per_km
    total_distance = battery_capacity_kWh / adjusted_consumption_per_km
    estimated_duration = estimated_distance / avg_speed
    battery_drain_rate = energy_left / estimated_duration if estimated_duration != 0 else 0

    return {
        # "Predicted Energy Consumed (kWh) urm ciclu de utilizare": predicted_energy_consumed,
        "Estimated Distance Remaining (km)": estimated_distance,
        "Total Possible Distance (km) bateria incarcata": total_distance,
        "Estimated Time Remaining (hours)": estimated_duration,
        "Consum pe H (kWh/h)": battery_drain_rate
    }

def prepare_features_from_user_input(user_input):
    energy = user_input.get("Energy Consumed (kWh)", 20)
    rate = user_input["Charging Rate (kW)"]
    soc_start = user_input["State of Charge (Start %)"]
    soc_end = user_input["State of Charge (End %)"]
    dist = user_input["Distance Driven (since last charge) (km)"]
    temp = user_input["Temperature (°C)"]

    charge_diff = soc_end - soc_start if soc_end - soc_start != 0 else 1e-6

    features = user_input.copy()
    features["Charging Efficiency (kWh/h)"] = energy / rate if rate != 0 else 0
    features["Energy per Charge %"] = energy / charge_diff
    features["Distance per kWh"] = dist / energy if energy != 0 else 0
    features["Total Charge Gained"] = charge_diff
    features["Charger Efficiency"] = rate / charge_diff
    features["Temperature Adjusted Consumption"] = energy * (1 + abs(temp - 20) / 20)

    
    return features

# === Încarcă modelul principal MLP pentru Energy Consumed ===
# model = tf.keras.models.load_model("mlp_ev_charging_model.h5", custom_objects={'mse': tf.keras.losses.MeanSquaredError()})

# === Încarcă modelele pentru estimare Km & Ore Rămase ===
model_km = tf.keras.models.load_model("modele/seq_layers_km_model.h5", custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
model_hours = tf.keras.models.load_model("modele/seq_layers_hours_model.h5", custom_objects={'mse': tf.keras.losses.MeanSquaredError()})

# === Încarcă datele și scalerul ===
df = pd.read_csv("ev_charging_synthetic_data.csv")
scaler_km_hours = joblib.load("modele/scaler_km_h.pkl")  # folosit pt. km & hours



# === Exemplu de input ===
user_input = {
    "Battery Capacity (kWh)": 50,
    "Energy Consumed (kWh)": 16,
    "Charging Rate (kW)": 34,
    "Time of Day": 14,
    "Day of Week": 3,
    "State of Charge (Start %)": 60,
    "State of Charge (End %)": 80,
    "Distance Driven (since last charge) (km)": 120,
    "Temperature (°C)": 18,
    "Vehicle Age (years)": 1,
    "Vehicle Model_BMW i3": 0, "Vehicle Model_Chevy Bolt": 0, "Vehicle Model_Hyundai Kona": 0,
    "Vehicle Model_Nissan Leaf": 0, "Vehicle Model_Tesla Model 3": 1,
    "Charging Station Location_Chicago": 0, "Charging Station Location_Houston": 1,
    "Charging Station Location_Los Angeles": 0, "Charging Station Location_New York": 0,
    "Charging Station Location_San Francisco": 0,
    "User Type_Casual Driver": 1, "User Type_Commuter": 0, "User Type_Long-Distance Traveler": 0,
    "Charger Type_DC Fast Charger": 1, "Charger Type_Level 1": 0, "Charger Type_Level 2": 0
}
print("Features folosite la fit:")
#print(scaler_km_hours.feature_names_in_)
# Presupunem că ai deja scalerul încărcat


full_features = prepare_features_from_user_input(user_input)
full_features_df = pd.DataFrame([full_features])
required_order = scaler_km_hours.feature_names_in_.tolist()
full_features_df = full_features_df.reindex(columns=required_order)
#print("full_features_df shape:", full_features_df.columns)

# Afișare predicții bazate pe modele
km_rem, hours_rem = predict_km_and_hours_remaining(full_features_df)
print(f"\n=== Estimări din modele salvate ===")
print(f"Km rămași estimați (MLP): {km_rem} km")
print(f"Ore rămase estimate (MLP): {hours_rem} ore")

# Estimări bazate pe logică calculată
result = predict_distance_and_time(100, 80, 1, 18, 'Autostrada', full_features)
print(f"\n=== Estimări pe bază de logică economică ===")
for k, v in result.items():
    print(f"{k}: {v:.2f}")


Features folosite la fit:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step

=== Estimări din modele salvate ===
Km rămași estimați (MLP): 717.8599853515625 km
Ore rămase estimate (MLP): 5.03000020980835 ore

=== Estimări pe bază de logică economică ===
Estimated Distance Remaining (km): 871.46
Total Possible Distance (km) bateria incarcata: 871.46
Estimated Time Remaining (hours): 8.71
Consum pe H (kWh/h): 9.18
